# John Maeda: Quickstart Your RAGventure With Agents

## 🔥 Let's get the required packages

In [ ]:
#!import ../config/MaedaSettings.cs 
#!import ../config/Utils.cs

#r "nuget: Azure.AI.OpenAI, 2.0.0-beta.4"
#r "nuget: Azure.Core, 1.42.0"
#r "nuget: Azure.Identity, 1.13.0-beta.1"
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.18.0-rc"
#r "nuget: Microsoft.SemanticKernel.Abstractions, 1.18.1-rc"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.18.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, 1.18.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.18.1-rc"
#r "nuget: Microsoft.SemanticKernel.Core, 1.18.1-rc"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.18.1-alpha"
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"

My config.json looks like this:
```
{
  "type": "azure",
  "model": "gpt-4o",
  "endpoint": "https://ai-johnmaeda.../",
  "apikey": "...",
  "org": "",
  "asst1": "asst_...",
  "asst2": "asst_...",
  "AZURE_SPEECH_KEY": "489....",
  "AZURE_SPEECH_REGION": "eastus"
}
```

In [ ]:
#!import ../config/MaedaSettings.cs
#!import ../config/Utils.cs

var settingsPath = Path.GetFullPath("../config/settings.json");
Console.WriteLine($"Settings Path: {settingsPath}");

var maedaSettings = new MaedaSettings(settingsPath, true);

// Load all settings into a tuple
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) 
    = maedaSettings.LoadSettings();

# Let's have a plugin available

In [95]:
using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Plugins.Core;
using System;
using System.Text.Json;
using System.Collections.Generic;
using System.Threading.Tasks;
using System.ComponentModel;
using System.Net.Http;
using System.Net.Http.Headers;
using Microsoft.Extensions.DependencyInjection;
using System.Threading;
using OpenAI.Files;
using OpenAI.VectorStores;

public class TimePlugin
{
    [KernelFunction]
    [Description("Retrieves the current time in UTC.")]
    public string GetCurrentUtcTime() => DateTime.UtcNow.ToString("R");
}

# Then we create an AI agent that uses the tool

In [103]:
#!import ../config/Utils.cs

#pragma warning disable SKEXP0110, SKEXP0001, SKEXP0050, CS8600, CS8604, OPENAI001

public class MyAzureAIAgent
{
    private readonly string _assistantId;
    private readonly string _deploymentName;
    private readonly string _endpoint;
    private readonly string _apiKey;
    private string? _threadId;
    private OpenAIAssistantAgent? _assistantAgent;
    public string namae;

    public MyAzureAIAgent(string deploymentName, string endpoint, string apiKey, string assistantId)
    {
        _assistantId = assistantId;
        _deploymentName = deploymentName;
        _endpoint = endpoint;
        _apiKey = apiKey;
    }

    public OpenAIAssistantAgent GetAgent()
    {
        return _assistantAgent;
    }
    
    public async Task InitializeAgent()
    {
        IKernelBuilder builder = Microsoft.SemanticKernel.Kernel.CreateBuilder();
        builder.AddAzureOpenAIChatCompletion(
                        deploymentName: _deploymentName,
                        endpoint: _endpoint,
                        apiKey: _apiKey);
        builder.Plugins.AddFromType<TimePlugin>();
        Microsoft.SemanticKernel.Kernel kernel = builder.Build();

        OpenAIClientProvider provider = OpenAIClientProvider.ForAzureOpenAI(_apiKey, new Uri(_endpoint));

        _assistantAgent = await OpenAIAssistantAgent.RetrieveAsync(
            kernel,
            provider,
            _assistantId);
        
        namae = _assistantAgent.Name;

        // Create a thread for the agent interaction.
        _threadId = await _assistantAgent.CreateThreadAsync();
        Console.WriteLine($"Agent {namae} with ID {Utils.ObfuscateString(_assistantId)} initialized.");
    }

    public async Task<string> AskAgent(string question)
    {
        if (_assistantAgent == null || _threadId == null)
        {
            throw new InvalidOperationException("Agent not initialized. Call InitializeAgent first.");
        }

        // Add the user's question to the chat.
        await _assistantAgent.AddChatMessageAsync(_threadId, new ChatMessageContent(AuthorRole.User, question));

        // Retrieve the response.
        string responseText = "";
        await foreach (ChatMessageContent content in _assistantAgent.InvokeAsync(_threadId))
        {
            if (content.Role != AuthorRole.Tool)
            {
                responseText += content.Content;
            }
        }

        return responseText; // Return the response as a string
    }

    public async Task EndSession()
    {
        if (_assistantAgent != null && _threadId != null)
        {
            bool result = await _assistantAgent.DeleteThreadAsync(_threadId);
            if (result)
            {
                _threadId = null;
            }
            else
            {
                Console.WriteLine("Failed to delete the thread.");
            }
        }
    }
}


In [97]:
#pragma warning disable SKEXP0110, SKEXP0001, SKEXP0050, CS8600, CS8604, OPENAI001

public string CreateAtName(string name)
{
    return name.Replace(" ", "").ToLower();
}

public async Task RunIndependentAgentSession(List<string> assistantIds, string model, string azureEndpoint, string apiKey)
{
    // Dictionary to hold initialized agents (mapped by processed agent names)
    var agents = new Dictionary<string, MyAzureAIAgent>(StringComparer.OrdinalIgnoreCase);
    string? defaultAgentName = null;

    try
    {
        // Instantiate and initialize each agent from the list of assistant IDs
        foreach (var assistantId in assistantIds)
        {
            var agent = new MyAzureAIAgent(model, azureEndpoint, apiKey, assistantId);
            await agent.InitializeAgent();

            // Process the namae to remove spaces and lowercase it
            string processedNamae = CreateAtName(agent.namae);

            // Add agent to dictionary with the processed name as the key
            agents[processedNamae] = agent;

            // Set the first agent as the default if it's not already set
            if (defaultAgentName == null)
            {
                defaultAgentName = processedNamae;
            }
        }

        // Ensure we have a default agent
        if (defaultAgentName == null)
        {
            throw new InvalidOperationException("No default agent found. Please check your assistant IDs.");
        }

        string input;
        do
        {
            // Print available agent names with an at sign in front of each of them and all on a single line
            // accrue them and include the at sign in fron tof them
            string availableAgents = "";
            foreach (var agentName in agents.Keys)
            {
                availableAgents += $"@{agentName} ";
            }

            // Get user input
            input = await InteractiveKernel.GetInputAsync($"Ask {availableAgents} (type '/bye' to exit):");

            if (input.ToLower() != "/bye")
            {
                // By default, use the default agent
                string selectedAgentName = defaultAgentName;

                // Check if there's an @mention in the input
                foreach (var agentName in agents.Keys)
                {
                    if (input.ToLower().Contains($"@{agentName}"))
                    {
                        selectedAgentName = agentName;

                        // Remove the @mention from the input
                        input = input.Replace($"@{agentName}", "").Trim();
                        break;
                    }
                }

                // Get the selected agent
                var selectedAgent = agents[selectedAgentName];
                string response = await selectedAgent.AskAgent(input);
                Console.WriteLine($"@{selectedAgentName}: {response}");
            }

        } while (input.ToLower() != "/bye");
    }
    finally
    {
        // Clean up and end each agent's session
        foreach (var agent in agents.Values)
        {
            await agent.EndSession();
        }
    }
}


# Make a kernel

In [98]:
#pragma warning disable SKEXP0110, SKEXP0001, SKEXP0050, CS8600, CS8604, OPENAI001

IKernelBuilder builder = Microsoft.SemanticKernel.Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion(
                deploymentName: model,
                endpoint: azureEndpoint,
                apiKey: apiKey);

Microsoft.SemanticKernel.Kernel kernel = builder.Build();

OpenAIClientProvider provider = OpenAIClientProvider.ForAzureOpenAI(apiKey, new Uri(azureEndpoint));

In [ ]:
#pragma warning disable SKEXP0110, SKEXP0001, SKEXP0050, CS8600, CS8604, OPENAI001

// Call the session runner and pass the list of assistant IDs inline
await RunIndependentAgentSession(
    new List<string>
    {
        maedaSettings.ReadString("asst1"),
        maedaSettings.ReadString("asst2"),
        // Add more assistant IDs here if needed
    },
    model, 
    azureEndpoint, 
    apiKey
);


In [ ]:
#!import ../config/Utils.cs
#!import ../config/MaedaAgentManager.cs

MaedaAgentManager agentManager = new MaedaAgentManager(kernel, provider);

string agentName = "Keanu Jeevs";
string description = "Hipster Matrix Fan";
string instructions = "Keanu speaks like he's from Bill and Ted's Excellent Adventure and also like Neo from the Matrix.";
List<string> filePaths = new List<string> { "travelinfo.txt" };
string agentId = await agentManager.CreatePlainAgent(agentName, model, description, instructions);
Console.WriteLine($"The agent {Utils.ObfuscateString(agentId)} has been minted. Refer to @{CreateAtName(agentName)} in chat.");

In [ ]:
#pragma warning disable SKEXP0110, SKEXP0001, SKEXP0050, CS8600, CS8604, OPENAI001

// Call the session runner and pass the list of assistant IDs inline
await RunIndependentAgentSession(
    new List<string>
    {
        settings.ReadString("asst1"),
        settings.ReadString("asst2"),
        // Add more assistant IDs here if needed
        agentId
    },
    model, 
    azureEndpoint, 
    apiKey
);


In [ ]:
#!import ../config/Utils.cs
#!import ../config/MaedaAgentManager.cs

string agentName = "John Traveler";
string description = "Travel agent";
string instructions = "John is ready to answer questions and assist with file-related queries.";
List<string> filePaths = new List<string> { "travelinfo.txt" };
string agentId = await agentManager.CreateAgentWithFilesAsync(agentName, model, description, instructions, filePaths);
Console.WriteLine($"The agent {Utils.ObfuscateString(agentId)} has been minted. Refer to @{CreateAtName(agentName)} in chat.");
Console.WriteLine($"Warning: The agent {agentId} has files and a vector store attached to it that may incur a running cost if not deleted.");

In [ ]:
#pragma warning disable SKEXP0110, SKEXP0001, SKEXP0050, CS8600, CS8604, OPENAI001

// Call the session runner and pass the list of assistant IDs inline
await RunIndependentAgentSession(
    new List<string>
    {
        maedaSettings.ReadString("asst1"),
        maedaSettings.ReadString("asst2"),
        // Add more assistant IDs here if needed
        agentId
    },
    model, azureEndpoint, apiKey
);

In [ ]:
#!import ../config/MaedaAgentManager.cs

await agentManager.DeleteAssociatedAgentFilesAndVectorStoreAsync(agentId);

In [ ]:
#!import ../config/MaedaAgentManager.cs

await agentManager.DeleteAllAgentsAsync();

In [113]:
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.Agents.OpenAI;

#pragma warning disable SKEXP0110, SKEXP0001, SKEXP0050, CS8600, CS8604, OPENAI001

public async Task RunMixedAgentSession(List<string> assistantIds, string model, string azureEndpoint, string apiKey)
{
    // Dictionary to hold initialized agents (mapped by processed agent names)
    var agents = new Dictionary<string, MyAzureAIAgent>(StringComparer.OrdinalIgnoreCase);
    string? defaultAgentName = null;

    try
    {
        // Instantiate and initialize each agent from the list of assistant IDs
        foreach (var assistantId in assistantIds)
        {
            var agent = new MyAzureAIAgent(model, azureEndpoint, apiKey, assistantId);
            await agent.InitializeAgent();

            // Process the namae to remove spaces and lowercase it
            string processedNamae = CreateAtName(agent.namae);

            // Add agent to dictionary with the processed name as the key
            agents[processedNamae] = agent;

            // Set the first agent as the default if it's not already set
            if (defaultAgentName == null)
            {
                defaultAgentName = processedNamae;
            }
        }

        // Ensure we have a default agent
        if (defaultAgentName == null)
        {
            throw new InvalidOperationException("No default agent found. Please check your assistant IDs.");
        }

 // Create a chat for agent interaction.
        OpenAIAssistantAgent one = agents["sun"].GetAgent();
        OpenAIAssistantAgent two = agents["sarah"].GetAgent();
        AgentGroupChat chat =
            new(one, two)
            {
                ExecutionSettings =
                    new()
                    {
                        // Here a TerminationStrategy subclass is used that will terminate when
                        // an assistant message contains the term "approve".
                        TerminationStrategy =
                            new ApprovalTerminationStrategy()
                            {
                                Agents = [one],
                                MaximumIterations = 3,
                            }
                    }
            };

        // Invoke chat and display messages.
        ChatMessageContent input = new(AuthorRole.User, "Talk about technology in the age of AI. When you agree with the other's opinion say approve.");
        chat.AddChatMessage(input);
        Console.WriteLine(input);

        await foreach (ChatMessageContent response in chat.InvokeAsync())
        {
            // write the originator
            Console.WriteLine($"{response.AuthorName}: {response}");
        }

        Console.WriteLine($"\n[IS COMPLETED: {chat.IsComplete}]");

    }
    finally
    {
        // Clean up and end each agent's session
        foreach (var agent in agents.Values)
        {
            await agent.EndSession();
        }
    }
}

public class ApprovalTerminationStrategy : TerminationStrategy
{
    // Terminate when the final message contains the term "approve"
    protected override Task<bool> ShouldAgentTerminateAsync(Agent agent, IReadOnlyList<ChatMessageContent> history, CancellationToken cancellationToken)
        => Task.FromResult(history[history.Count - 1].Content?.Contains("approve", StringComparison.OrdinalIgnoreCase) ?? false);
}


In [ ]:
#pragma warning disable SKEXP0110, SKEXP0001, SKEXP0050, CS8600, CS8604, OPENAI001

// Call the session runner and pass the list of assistant IDs inline
await RunMixedAgentSession(
    new List<string>
    {
        maedaSettings.ReadString("asst1"),
        maedaSettings.ReadString("asst2"),
        // Add more assistant IDs here if needed
    },
    model, azureEndpoint, apiKey
);

# We did it!

## 🧑‍🍳 Let's get cooking on this problem together!

### 👁️ We may use some image recognition in this example

![](caikwithabhijit.jpg)

![](https://oaidalleapiprodscus.blob.core.windows.net/private/org-rocrupyvzgcl4yf25rqq6d1v/user-llcMA8baSvN14XBWssakSbKG/img-0EmlTr3CY5EPQ5GTb9LqLNMd.png?st=2024-02-09T20%3A24%3A56Z&se=2024-02-09T22%3A24%3A56Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-02-09T20%3A54%3A28Z&ske=2024-02-10T20%3A54%3A28Z&sks=b&skv=2021-08-06&sig=W0ZV3kQMYePF4z/8WPmYp1uhXuL27x1t7BAUlAfoPVA%3D)

![](https://oaidalleapiprodscus.blob.core.windows.net/private/org-rocrupyvzgcl4yf25rqq6d1v/user-llcMA8baSvN14XBWssakSbKG/img-IS4fYpZLQxIhReOe7419D8tV.png?st=2024-02-09T16%3A14%3A57Z&se=2024-02-09T18%3A14%3A57Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-02-08T21%3A23%3A05Z&ske=2024-02-09T21%3A23%3A05Z&sks=b&skv=2021-08-06&sig=9itiXCvmkMwKCq71dWCk/yISgWFr9pECL92z7skME1c%3D)

### 🙉 Let's add the auditory modalities

In [ ]:
#r "nuget: Microsoft.CognitiveServices.Speech, 1.34.1"
#r "nuget: NetCoreAudio, 1.8.0"
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.EnvironmentVariables, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.UserSecrets, 8.0.0"

#!import ../config/MaedaSettings.cs
#!import ../config/AzureSpeech.cs

string subscriptionKey = Settings.ReadString("AZURE_SPEECH_KEY");
string subscriptionRegion = Settings.ReadString("AZURE_SPEECH_REGION");

var speechService = new SpeechRecognitionService(subscriptionKey, subscriptionRegion);

### 🎤 Let's listen up

In [ ]:
string recognizedText = await speechService.RecognizeOnceAsync();
Console.WriteLine($"Transcribed: {recognizedText}");

### 🗣️ And let's speak up

In [90]:
#pragma warning disable SKEXP0101

await speechService.SynthesizeSpeechAsync(recognizedText);

## 🔥 We fire up Semantic Kernel's Experimental Agents ...

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;
using Microsoft.SemanticKernel.Experimental.Agents;

const string OpenAIFunctionEnabledModel = "gpt-4-1106-preview";


In [ ]:
using System.IO;
using YamlDotNet.Serialization;
using YamlDotNet.Serialization.NamingConventions;

using System.IO;
using YamlDotNet.Serialization;
using YamlDotNet.Serialization.NamingConventions;
using System;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;
using Microsoft.SemanticKernel.Experimental.Agents;

#pragma warning disable SKEXP0101

public class Personay
{
    public string Name { get; set; }
    public string Instructions { get; set; }
    public string Description { get; set; }
}

public class NameGenerator
{
    private List<string> names;
    private int currentIndex = -1;

    public NameGenerator()
    {
        // Initialize the list with ungendered names
        names = new List<string>
        {
            "Alex", "Jordan", "Taylor", "Morgan", "Casey",
            "Riley", "Jamie", "Avery", "Reese", "Skyler",
            "Quinn", "Peyton", "Cameron", "Sawyer", "Drew",
            "Charlie", "Emerson", "Dakota", "Parker", "Sidney"
        };
    }

    public string GetNextName()
    {
        // Increment the index and reset if it exceeds the list count
        currentIndex = (currentIndex + 1) % names.Count;
        return names[currentIndex];
    }
}

 // Track agents for clean-up
static readonly Dictionary<string, IAgent> s_agents = new();

IAgentThread? s_currentThread = null;

async Task<IAgent> CreateAgentAsync(string name, string instructions, string description)
{
    var agent = await new AgentBuilder()
                    .WithOpenAIChatCompletion(OpenAIFunctionEnabledModel, apiKey)
                    .WithInstructions(instructions)
                    .WithName(name)
                    .WithDescription(description)
                    .BuildAsync();

    return Track(name, agent);
}

async Task CleanUpAsync()
{
    Console.WriteLine("🧽 Cleaning up ...");

    if (s_currentThread != null)
    {
        Console.WriteLine("Thread going away ...");
        s_currentThread.DeleteAsync();
        s_currentThread = null;
    }
    
    if (s_agents.Any())
    {
        Console.WriteLine("Agents going away ...");
        await Task.WhenAll(s_agents.Values.Select(agent => agent.DeleteAsync()));
        s_agents.Clear();
    }
}

IAgent Track(string name, IAgent agent)
{
    s_agents[name] = agent; // Add or update the agent in the dictionary
    return agent;
}



### 🐣 Let's hatch the agent(s)

In [ ]:
List<string> personasAvailable = [
    // front of house
    "RestaurantHost",
    "RestaurantServer",
    "RestaurantBussing",
    // back of house
    "RestaurantOrderManager",
    "RestaurantCook",
    "RestaurantKitchenPrep"];

List<(string Name, string Instructions, string Description)> agentInfo = new();
NameGenerator nameGenerator = new NameGenerator();

var rollcall = "This is the rollcall for the restaurant:\n";

foreach (var (a, i) in personasAvailable.Select((value, idx) => (value, idx + 1)))
{
    var yaml = File.ReadAllText($"../agents/{a}.yaml");
    var deserializer = new DeserializerBuilder()
        .WithNamingConvention(CamelCaseNamingConvention.Instance) // Use camel case naming convention
        .Build();
    var p = deserializer.Deserialize<Personay>(yaml);
    string fakeName = nameGenerator.GetNextName();
    string desc = $"{p.Name}: {p.Description}";
    string instr = p.Instructions;
    agentInfo.Add((fakeName, instr, desc));
    Console.WriteLine($"Agent defined: {fakeName} --> {desc}");
    rollcall += $"- {i}) {fakeName} is the {desc}\n";
}

foreach (var (name, instructions, description) in agentInfo)
{
    await CreateAgentAsync(name, instructions, description);
}

Console.WriteLine(rollcall);
s_agents

![](serviceoverview.jpg)
![](simplifiedjourney.jpg)
![](serviceblueprint.jpg)

### 🕵️💨 We check which agent will be relevant if a customer's arrived

In [ ]:
#pragma warning disable SKEXP0101

IAgentThread? thread = null;

thread = await s_agents["Alex"].NewThreadAsync();
await thread.AddUserMessageAsync(
    $"{rollcall}. A customer has arrived at the restaurant. Who is up?");

var agentMessages = await thread.InvokeAsync(s_agents["Alex"]).ToArrayAsync();

Console.WriteLine(Utils.WordWrap(agentMessages[0].Content, 80));

### 🕵️💨 And then we give the agent a whirl

In [ ]:
#pragma warning disable SKEXP0101

var whichAgent = "Alex";

thread = await s_agents[whichAgent].NewThreadAsync();
await thread.AddUserMessageAsync(
    $"Let's pretend a customer has arrived at the restaurant. Please do your thing.");

Console.WriteLine("🎤 As the customer, say something to the agent ...");

string recognizedText = await speechService.RecognizeOnceAsync();
await thread.AddUserMessageAsync(recognizedText);

var agentMessages = await thread.InvokeAsync(s_agents[whichAgent]).ToArrayAsync();

Console.WriteLine(Utils.WordWrap(agentMessages[0].Content, 80));

await speechService.SynthesizeSpeechAsync(agentMessages[0].Content);

### 🕵️🗣️ Interact with this agent as you've just arrived

In [ ]:
#pragma warning disable SKEXP0101

Console.WriteLine("🎤 As the customer, say something to the agent ...");

string recognizedText = await speechService.RecognizeOnceAsync();
await thread.AddUserMessageAsync(recognizedText);

var agentMessages = await thread.InvokeAsync(s_agents[whichAgent]).ToArrayAsync();

Console.WriteLine(Utils.WordWrap(agentMessages[0].Content, 80));

await speechService.SynthesizeSpeechAsync(agentMessages[0].Content);

### 🧐 We'll next want to know which agent is needed next

In [ ]:
#pragma warning disable SKEXP0101

await thread.AddUserMessageAsync(
    $"{rollcall}. Which agent should be used next in this service scenario?");

var agentMessages = await thread.InvokeAsync(s_agents[whichAgent]).ToArrayAsync();

Console.WriteLine(Utils.WordWrap(agentMessages[0].Content, 80));

### 🕵️ You've got a new agent to interact with

In [ ]:
#pragma warning disable SKEXP0101

whichAgent = "Jordan";

await thread.AddUserMessageAsync(
    $"Say something to the guest while introducing yourself.");

var agentMessages = await thread.InvokeAsync(s_agents[whichAgent]).ToArrayAsync();

Console.WriteLine(Utils.WordWrap(agentMessages[0].Content, 80));

await speechService.SynthesizeSpeechAsync(agentMessages[0].Content, "en-US-BrianNeural");

### 🕵️🗣️ Interact with this agent so we can continue down the journey

In [ ]:
#pragma warning disable SKEXP0101

Console.WriteLine("🎤 As the customer, say something to the agent ...");

string recognizedText = await speechService.RecognizeOnceAsync();
await thread.AddUserMessageAsync(recognizedText);

var agentMessages = await thread.InvokeAsync(s_agents[whichAgent]).ToArrayAsync();

Console.WriteLine(Utils.WordWrap(agentMessages[0].Content, 80));

await speechService.SynthesizeSpeechAsync(agentMessages[0].Content, "en-US-BrianNeural");

In [ ]:
#pragma warning disable SKEXP0101

Console.WriteLine("🎤 As the customer, say something to the agent ...");

string recognizedText = await speechService.RecognizeOnceAsync();
await thread.AddUserMessageAsync(recognizedText);

var agentMessages = await thread.InvokeAsync(s_agents[whichAgent]).ToArrayAsync();

Console.WriteLine(Utils.WordWrap(agentMessages[0].Content, 80));

await speechService.SynthesizeSpeechAsync(agentMessages[0].Content, "en-US-BrianNeural");

### 🧐 We'll next want to know which agent is needed next

In [ ]:
#pragma warning disable SKEXP0101

await thread.AddUserMessageAsync(
    $"{rollcall}. Which agent should be used next in this service scenario?");

var agentMessages = await thread.InvokeAsync(s_agents[whichAgent]).ToArrayAsync();

Console.WriteLine(Utils.WordWrap(agentMessages[0].Content, 80));
await speechService.SynthesizeSpeechAsync(agentMessages[0].Content, "en-US-BrianNeural");